# Processing the train and test data to put each question as a single line and then use gensim to generate word embeddings

In [2]:
import pandas as pd
import numpy as np
import os
import sys

In [56]:
from tqdm import tqdm_notebook

In [3]:
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence

In [44]:
from nltk.tokenize import TreebankWordTokenizer 

In [69]:
EMBEDDING_DIM = 300

In [45]:
tokenize = TreebankWordTokenizer().tokenize

In [4]:
data_dir = "/Users/roopal/workspace/kaggle/quora_question_pairs/data"

In [48]:
data_file = os.path.join(data_dir, "qqp_text")
data_file_tokens = os.path.join(data_dir, "qqp_text_tokens")

In [5]:
train_file = os.path.join(data_dir, "train.csv.pkl")
test_file = os.path.join(data_dir, "test.csv.pkl")

In [72]:
w2v_model = os.path.join(data_dir, "w2v.model")
w2v_model_bin = os.path.join(data_dir, "w2v.model.bin")
w2v_vocab = os.path.join(data_dir, "w2v.model.vocab")

In [8]:
df_train = pd.read_csv(train_file, index_col=False)

In [9]:
df_test = pd.read_csv(test_file, index_col=False)

In [10]:
print df_train.columns
print df_test.columns

Index([u'id', u'qid1', u'qid2', u'question1', u'question2', u'is_duplicate'], dtype='object')
Index([u'test_id', u'question1', u'question2'], dtype='object')


In [32]:
df_questions = df_train['question1']

In [33]:
type(df_questions)

pandas.core.series.Series

In [34]:
len(df_questions)

404290

In [37]:
df_questions = df_questions.append(df_train["question2"])

In [38]:
len(df_questions)

808580

In [39]:
df_questions = df_questions.append(df_test["question1"])

In [40]:
len(df_questions)

3154376

In [41]:
df_questions = df_questions.append(df_test["question2"])

In [42]:
len(df_questions)

5500172

In [49]:
df_questions.to_csv(data_file, index=False)

In [61]:
def tokenize_text(txt_file, txt_tokens_file):
    with open(txt_tokens_file, 'w') as out:
        with open(txt_file, 'r') as inp:
            data = inp.readlines()
            pbar = tqdm_notebook(total=len(data), desc="current_line", leave=False)
            for i, text in enumerate(data):
                tokenized_text = ' '.join(tokenize(text))
                out.write(tokenized_text + "\n")
                pbar.update()
            pbar.close()

In [62]:
tokenize_text(data_file, data_file_tokens)

In [74]:
def generate_vectors(input_filename, output_filename_model, output_filename_model_w2v_bin, vocab_filename):
    sentences = LineSentence(input_filename)
#     print sentences
#     bigrams = Phrases(sentences, min_count=5, threshold=10)
#     print bigrams
#     trigrams = Phrases(bigrams[sentences],min_count=5, threshold=10)
#     print trigrams
    print "generating models"
    model = Word2Vec(
        sentences, size=EMBEDDING_DIM, window=5, min_count=5, workers=7, iter=6
    )
    print "saving model"
    model.save(output_filename_model)
    print "saving bin model and vocab"
    model.wv.save_word2vec_format(output_filename_model_w2v_bin, vocab_filename, binary=True)

In [ ]:
generate_vectors(data_file_tokens, w2v_model, w2v_model_bin, w2v_vocab)

generating models


In [ ]:
print "Done..."